# Ejemplo de IA generativa:

Mediante una red neuronal simple se genera texto

In [1]:
# Importación de bibliotecas y librerías
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import requests
import os
import pandas as pd
import re
import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

## Paso 1: Obtener y preprocesar el texto

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

path = "pg4300.txt"
with open(path) as f:
    text = f.read()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/n.c.rodriguez/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/n.c.rodriguez/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# Preprocesar texto (simplificado, considerar limpiar encabezados y pies de página del sitio)
text = text.lower()  # Convertir a minúsculas
# Asumiendo que 'text' es tu texto completo
sentences = text.split('.')  # Esto divide el texto en frases basándose en el punto.

In [4]:
# Crear un DataFrame con las frases
df_sentences = pd.DataFrame(sentences, columns=['sentence'])

In [19]:
df_sentences.head()

,sentence,length
0,"﻿stately, plump buck mulligan came from the st...",121
1,"a yellow\ndressinggown, ungirdled, was sustain...",89
2,he held the bowl aloft and intoned:\n\n—_intro...,62
3,"halted, he peered down the dark winding stairs...",174
4,he faced about\nand blessed gravely thrice the...,99


In [5]:
# Eliminar espacios en blanco y filtrar frases vacías
df_sentences['sentence'] = df_sentences['sentence'].str.strip()
df_sentences = df_sentences[df_sentences['sentence'] != '']
#longitud en caracteres
df_sentences['length'] = df_sentences['sentence'].str.len()
average_length = df_sentences['length'].mean()
median_length = df_sentences['length'].median()
print("Longitud promedio de las frases:", average_length)
print("Longitud mediana de las frases:", median_length)

Longitud promedio de las frases: 68.61963819470347
Longitud mediana de las frases: 36.0


In [6]:
# Eliminar caracteres especiales y números
text = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑ ]', '', text)
# Normalizar acentos
text = unidecode.unidecode(text)

In [7]:
# Tokenización
words = word_tokenize(text)

In [8]:
# Eliminar palabras de parada
filtered_words = [word for word in words if word not in stopwords.words('english')]

In [9]:
# Reconstruir el texto
cleaned_text = ' '.join(filtered_words)

In [10]:
chars = sorted(list(set(cleaned_text)))  # Listar caracteres únicos
char_to_int = dict((c, i) for i, c in enumerate(chars))  # Crear diccionario de caracteres a índices

In [11]:
# Crear secuencias de caracteres y sus próximos caracteres
maxlen = 50
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Número de secuencias:', len(sentences))

Número de secuencias: 474818


In [12]:
# Vectorizar las secuencias de entrada y etiquetas
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_int[char]] = 1
    y[i, char_to_int[next_chars[i]]] = 1

## Paso 2: Construir el modelo RNN

In [13]:
model = Sequential()

In [14]:
# Primera capa LSTM con return_sequences=True para apilar otra capa LSTM después
model.add(LSTM(128, return_sequences=True, input_shape=(maxlen, len(chars))))
# Opcionalmente, agrega Dropout para regularización
model.add(tf.keras.layers.Dropout(0.2))

In [15]:
# Segunda capa LSTM
model.add(LSTM(128))
# Otra capa de Dropout
model.add(tf.keras.layers.Dropout(0.2))

In [16]:
# Capa de salida
model.add(Dense(len(chars), activation='softmax'))

In [17]:
# Compilación del modelo
model.compile(loss='categorical_crossentropy', optimizer='RMSprop')

## Paso 3: Entrenar el modelo

In [18]:
model.fit(x, y, batch_size=64, epochs=10)  # Ajusta el número de épocas según sea necesario
model.save('ulisses_0.2_token.keras')

Epoch 1/10
2909/7420 [==========>...................] - ETA: 3:56 - loss: 2.3677

KeyboardInterrupt: 